In [ ]:
!pip install timm # install pytorch image models
!pip install torchmetrics

In [ ]:
import torch
import pandas as pd
import torchvision.models as models
import timm
import albumentations as A
import cv2
import numpy as np
import tensorflow as tf

from torch import nn
from  torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import transforms

In [ ]:
class CustomModel(torch.nn.Module): 
    def __init__(self, model_backbone):
        super(CustomModel,self).__init__()
        self.model = model_backbone
        self.num_in_features = self.model.get_classifier().in_features
        print(self.num_in_features)
        self.model.classifier = nn.Sequential(
            nn.BatchNorm1d(self.num_in_features),
            nn.Linear(self.num_in_features, 512),
            nn.Dropout(0.5),
            nn.ReLU(inplace=True),
            nn.Linear(512, 100),
        )
    def forward(self,x):
        x = self.model(x)
        return x

In [ ]:
class SorghumDataset(Dataset):
    def __init__(self, dirs, labels, transformation=None):
        super(SorghumDataset,self).__init__()
        self.dirs = dirs
        self.labels = labels
        self.transformation = transformation
    def __len__(self):
        return len(self.dirs)

    def __getitem__(self, index):
        image = cv2.imread(self.dirs[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[index] # need to one hot encoding here
        
        image = np.array(image)

        if self.transformation:
            aug_image = self.transformation(image=image)
            image = aug_image['image']
            
        image = image / 255.
        image = image.transpose((2, 0, 1))
        
        image = torch.from_numpy(image).type(torch.float32)
        image = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
        
        labels = torch.from_numpy(np.array(self.labels[index])).type(torch.float32)


        return image, labels

# Resnet

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name = 'resnext50d_32x4d'

backbone = timm.create_model(model_name,pretrained=True)
model = CustomModel(backbone)

model.to(device)

In [ ]:
checkpoint = torch.load('../input/sorghum-efficientnetv2-0-846-private-lb/resnext50d_32x4d_best.pt')
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')
sub.head()

In [ ]:
sub["filename"] = sub["filename"].apply(lambda image: '../input/sorghum-id-fgvc-9/test/' + image)
sub["cultivar"] = 0
sub.head()

In [ ]:
validation_transformation = A.Compose([
    A.Resize(width=512, height=512, p=1.0)
])

testing_dataset = SorghumDataset(sub['filename'], sub['cultivar'], validation_transformation)
testing_dataloader = DataLoader(testing_dataset, 
                                batch_size=32, 
                                shuffle=False, 
                                num_workers=1)

In [ ]:
predictions = []
cnt = 0

resnet_preds = []

with torch.no_grad():
    for image, label in tqdm(testing_dataloader):
        image = image.to(device)
        outputs = model(image)
        for i in range(len(outputs)):
            resnet_preds.append(outputs[i][:100])
#         resnet_preds.append(outputs[0][:100])
        preds = outputs.detach().cpu()
        predictions.append(preds.argmax(1))

In [ ]:
resnet_preds_ = []

for i in range(len(resnet_preds)):
    resnet_preds_.append(resnet_preds[i].tolist())

In [ ]:
resnet_preds_list = pd.DataFrame({'0_class':[]})

for i in range(99, 0, -1): 
    resnet_preds_list.insert(1, "{}_class".format(i), [])

for i in range(len(resnet_preds_)):
    resnet_preds_list.loc[i] = resnet_preds_[i]

In [ ]:
resnet_preds_list.to_csv('resnet_submission.csv', index=False)

resnet_preds_list.head()

# EfficientNetB5

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name = 'tf_efficientnetv2_m_in21k'

backbone = timm.create_model(model_name,pretrained=True)
B5_model = CustomModel(backbone)

B5_model.to(device)

In [ ]:
checkpoint = torch.load('../input/sorghum-identification-12345/tf_efficientnetv2_m_in21k_best.pt')
B5_model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
predictions = []
cnt = 0

B5_preds = []

with torch.no_grad():
    for image, label in tqdm(testing_dataloader):
        image = image.to(device)
        outputs = model(image)
        for i in range(len(outputs)):
            B5_preds.append(outputs[i][:100])
#         resnet_preds.append(outputs[0][:100])
        preds = outputs.detach().cpu()
        predictions.append(preds.argmax(1))

In [ ]:
B5_preds_ = []

for i in range(len(B5_preds)):
    B5_preds_.append(B5_preds[i].tolist())

In [ ]:
B5_preds_list = pd.DataFrame({'0_class':[]})

for i in range(99, 0, -1): 
    B5_preds_list.insert(1, "{}_class".format(i), [])

for i in range(len(B5_preds_)):
    B5_preds_list.loc[i] = B5_preds_[i]

In [ ]:
B5_preds_list

In [ ]:
B5_preds_list.to_csv('B5_submission.csv', index=False)

B5_preds_list.head()

# EfficientNetB4

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

model_B4 = EfficientNet.from_name('efficientnet-b4')
model_B4.load_state_dict(torch.load("../input/test-for-kaggle-0426/epoch25.pt", map_location='cuda'))

model_B4.to(device)

In [ ]:
predictions = []
cnt = 0

B4_preds = []

with torch.no_grad():
    for image, label in tqdm(testing_dataloader):
        image = image.to(device)
        outputs = model_B4(image)
        for i in range(len(outputs)):
            B4_preds.append(outputs[i][:100])
#         resnet_preds.append(outputs[0][:100])
        preds = outputs.detach().cpu()
        predictions.append(preds.argmax(1))

In [ ]:
B4_preds_ = []

for i in range(len(B4_preds)):
    B4_preds_.append(B4_preds[i].tolist())

In [ ]:
B4_preds_list = pd.DataFrame({'0_class':[]})

for i in range(99, 0, -1): 
    B4_preds_list.insert(1, "{}_class".format(i), [])

for i in range(len(B5_preds_)):
    B4_preds_list.loc[i] = B4_preds_[i]

In [ ]:
B4_preds_list.to_csv('B4_submission.csv', index=False)

B4_preds_list.head()

# Ensemble Learning (b4+b5+resnet)

In [ ]:
resnet_result = pd.read_csv('../input/resnet-submission/resnet_submission.csv')

resnet_result.head()

In [ ]:
b5_result = pd.read_csv('../input/b5-sub/B5_submission.csv')

b5_result = b5_result * 10

b5_result.head()

In [ ]:
b4_result = pd.read_csv('../input/b4-sub/B4_submission.csv')

# b4_result = b4_result * 10

b4_result.head()

In [ ]:
b5_result['max_value'] = b5_result.max(axis=1)
b5_result['class'] = b5_result.idxmax(axis=1)

resnet_result['max_value'] = resnet_result.max(axis=1)
resnet_result['class'] = resnet_result.idxmax(axis=1)

b4_result['max_value'] = b4_result.max(axis=1)
b4_result['class'] = b4_result.idxmax(axis=1)

In [ ]:
b5_result.head()

In [ ]:
resnet_result.head()

In [ ]:
b4_result.head()

In [ ]:
result = []

for i in range(len(b5_result)):
    max_val = max((b5_result['max_value'][i] * 0.3),(resnet_result['max_value'][i] * 1.7),(b4_result['max_value'][i] * 0.5))
    
    if max_val == (b5_result['max_value'][i] * 0.3):
        result.append((max_val, b5_result['class'][i]))
        
    elif max_val == (resnet_result['max_value'][i] * 1.7):
        result.append((max_val, resnet_result['class'][i]))
        
    else:
        result.append((max_val, b4_result['class'][i]))

In [ ]:
result[:5]

In [ ]:
import re
 
tmp = []

for i in range(len(result)):
    tmp.append(int(re.sub(r'[^0-9]', '', result[i][1])))
    
print(tmp)

In [ ]:
df_all = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
df_all.dropna(inplace=True)

unique_cultivars = list(df_all["cultivar"].unique())

predictions = [unique_cultivars[pred] for pred in tmp]

In [ ]:
sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')
sub['cultivar'] = predictions
sub.to_csv('submission5.csv', index=False)
sub.head()